<a href="https://colab.research.google.com/github/will2bless1140/cpu-sheduling/blob/main/search_engine_for_publications_in_computational_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install BeautifulSoup
!pip install feedparser
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib import robotparser
!pip install json
import time
import json
from urllib.parse import urljoin
!pip install schedule

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Looking in indexes: https://pyp

In [ ]:
# to check the robot.txt rule of the site
import requests
from bs4 import BeautifulSoup
url = "https://pureportal.coventry.ac.uk/robots.txt"
page = requests.get(url)
print("robot.txt page info \n",page.text)


robot.txt page info 
 User-Agent: *
Crawl-Delay: 5
Disallow: /*?*format=rss
Disallow: /*?*export=xls
Sitemap: https://pureportal.coventry.ac.uk/sitemap.xml


In [ ]:
# to check if the website allows crawling
robotparser = robotparser.RobotFileParser()
robotparser.set_url("https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/")

# to read the robot.txt file
robotparser.read()
#to check user agent is allowed to access the page
if robotparser.can_fetch("my-bot", "/some-pages"):
  print("user is allowed to crawl")
else:
  print("user is not allowed to crawl")

user is allowed to crawl


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import schedule
import json

def crawl_published_data():
    url = "https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/"
    publications = []
    has_next_page = True

    while has_next_page:
        page = requests.get(url) # GET request to the URL
        soup = BeautifulSoup(page.text, "html.parser")

        publication_details = soup.find_all("div", class_="result-container") # finding all publication details on the page
        for details in publication_details:
            title = details.find("h3", class_="title") # finding the publication title element
            title_link = title.find("a", class_="link") # finding the link to the publication
            link = title_link.get("href") # extractig the link from the element

            pub = requests.get(link) # making a GET request to the publication link
            pub_soup = BeautifulSoup(pub.text, 'html.parser') # creating a BeautifulSoup object to parse the HTML content of the publication page
            abstracts = pub_soup.find("div", class_="rendering_researchoutput_abstractportal") # finding the element containing the publication abstract
            if abstracts:
                abstract = abstracts.find("div", class_="textblock").get_text(strip=True) # extracting the text content of the abstract element
            else:
                abstract = "" # if there is no abstract print nothing
            authors = []
            author_details = details.find_all('a', class_='link person') # finding all author details on the publication page
            authors_data = []
            for author in author_details:
                name = author.get_text().strip() # extracting the name of the author
                if name:
                    author_data = {"name": name, "href": author.get("href")} # storing the name and the link to the author page
                    authors_data.append(author_data)
            date = details.find("span", class_="date") # finding the date of publication
            publication = {
                    'title': title.get_text().strip(),
                    'link': link,
                    'authors': authors_data,
                    'date': date.get_text().strip(),
                    'abstract': abstract
                } # creating a dictionary containing the publication data
            publications.append(publication)

        next_page = soup.find("li", class_="next") # finding the link to the next page of publications
        if next_page is not None:
            url = urljoin(url, next_page.find("a")["href"]) # setting the URL to the link of the next page
        has_next_page = next_page is not None # checking if there is a next page
        time.sleep(5) # add a 5-second delay based on the robot.txt rule
    # save the output in JSON format
    with open("publication_data.json", "w") as f:
        json.dump(publications, f, indent=4)


    #viewing my output
    for publication in publications:
        print(publication['title'])
        print(publication['link'])
        if 'authors' in publication:
            for author in publication['authors']:
                print(author['name'], author['href'])
        if 'date' in publication:
            print(publication['date'])
        if 'abstract' in publication:
            print(publication['abstract'])
        print("\n")
    # sheduling my crawler to collect data again in the next one week
    print("crawling complete. next update in 1 week.")
schedule.every(1).week.do(crawl_published_data)

crawl_published_data()



An Extended Plant Circadian Clock Model for Characterising Flowering Time under Different Light Quality Conditions
https://pureportal.coventry.ac.uk/en/publications/an-extended-plant-circadian-clock-model-for-characterising-flower
Pay, M. L. https://pureportal.coventry.ac.uk/en/persons/miao-lin-pay
Christensen, J. https://pureportal.coventry.ac.uk/en/persons/jesper-christensen
He, F. https://pureportal.coventry.ac.uk/en/persons/fei-he
Roden, L. https://pureportal.coventry.ac.uk/en/persons/laura-roden
9 Jan 2023
Speed breeding has recently emerged as an innovative agricultural technology solution to meet the ever-increasing global food demand. In speed breeding, typically various light qualities (e.g., colour, duration, intensity) are modified to manipulate the circadian clock of the plants, which in turn alter the plant growth and enhance the productivity such as by reducing the flowering time. In order to develop a comprehensive framework describing plant growth, a model incorporating

In [ ]:
#this is viewing my jason saved file
with open('publication_data.json', 'r') as f:
            data = json.load(f)
print(data)

[{'title': 'An Extended Plant Circadian Clock Model for Characterising Flowering Time under Different Light Quality Conditions', 'link': 'https://pureportal.coventry.ac.uk/en/publications/an-extended-plant-circadian-clock-model-for-characterising-flower', 'authors': [{'name': 'Pay, M. L.', 'href': 'https://pureportal.coventry.ac.uk/en/persons/miao-lin-pay'}, {'name': 'Christensen, J.', 'href': 'https://pureportal.coventry.ac.uk/en/persons/jesper-christensen'}, {'name': 'He, F.', 'href': 'https://pureportal.coventry.ac.uk/en/persons/fei-he'}, {'name': 'Roden, L.', 'href': 'https://pureportal.coventry.ac.uk/en/persons/laura-roden'}], 'date': '9 Jan 2023', 'abstract': 'Speed breeding has recently emerged as an innovative agricultural technology solution to meet the ever-increasing global food demand. In speed breeding, typically various light qualities (e.g., colour, duration, intensity) are modified to manipulate the circadian clock of the plants, which in turn alter the plant growth and

In [ ]:
# checking the length
len(data)

221

In [ ]:

total_authors = 0
author_publications = {}

# Looping through the data to count authors and their publications
for publication in data:
    if 'authors' in publication:
        # Incrementing the total number of authors
        total_authors += len(publication['authors'])
        for author in publication['authors']:
            # Checking if the author already exists in the dictionary of author publications
            if author['name'] in author_publications:
                # Incrementing the count of publications for the author
                author_publications[author['name']] += 1
            else:
                # Adding a new author to the dictionary of author publications
                author_publications[author['name']] = 1

# Printing the total number of authors and their publications
print("Total number of authors:", total_authors)
for author, publications in author_publications.items():
    print(f"{author}: {publications} publications")


Total number of authors: 294
Pay, M. L.: 1 publications
Christensen, J.: 1 publications
He, F.: 15 publications
Roden, L.: 2 publications
Fanous, M.: 3 publications
Eden, J.: 1 publications
Daneshkhah, A.: 36 publications
Tariq, M.: 2 publications
Palade, V.: 47 publications
Ma, Y.: 7 publications
Jiang, X.: 8 publications
Ganeshamoorthy, S.: 1 publications
Klepl, D.: 6 publications
Eden, J. M.: 1 publications
Gunawardena, S. R. A. S.: 1 publications
Shah, H.: 3 publications
Hu, P.: 1 publications
Sadeghimanesh, A.: 10 publications
Nixon, J. D.: 4 publications
Gaura, E.: 14 publications
Halford, A.: 18 publications
Ajao-Olarinoye, M.: 1 publications
Al Bazi, A.: 3 publications
England, M.: 17 publications
Deo, A.: 3 publications
Azizsafaei, M.: 1 publications
Tart, M. S.: 4 publications
Pandey, A. K.: 1 publications
Brusey, J.: 11 publications
Babatunde, A.: 1 publications
Jess, B. J.: 1 publications
Grawemeyer, B.: 1 publications
Halloran, J.: 1 publications
Barket, R.: 2 publications

In [ ]:
# pre processing stage

In [ ]:
# creating stop words
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

sw = stopwords.words('english')
print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import json
import math
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

stemmer = PorterStemmer()

def preprocess(text):
    tokens = word_tokenize(text)
    #changeing to lowercase and checking alpha numeric
    token = [w.lower() for w in tokens if w.isalnum()]
    #removing stop words
    stopped_words = [w for w in token if w not in sw]
    stemming = [stemmer.stem(w) for w in stopped_words]
    return stemming


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# building the inverted index
def build_inverted_index(data):
    corpus = {}
    docid = 0


    for publication in data:
        # preprocessing the publication title
        tokens = preprocess(publication['title'])

        # looping through each token in the publication title
        for token in tokens:
            # checking if the token is in the corpus dictionary
            if token not in corpus:
                # if the token is not in the dictionary, add it with a count of 1 and initialize a document ID dictionary with a count of 1 for this document
                corpus[token] = {'number of appearance': 1, 'document_id': {docid: 1}}
            else:
                # if the token is already in the dictionary, increment the count of its appearances and the count of its appearances in this document
                if docid not in corpus[token]['document_id']:
                    corpus[token]['number of appearance'] += 1
                    corpus[token]['document_id'][docid] = 1
                else:
                    corpus[token]['document_id'][docid] += 1

        # increment the document ID after processing the current publication
        docid += 1

    return corpus

index = build_inverted_index(data)
index


{'extend': {'number of appearance': 1, 'document_id': {0: 1}},
 'plant': {'number of appearance': 2, 'document_id': {0: 1, 33: 1}},
 'circadian': {'number of appearance': 1, 'document_id': {0: 1}},
 'clock': {'number of appearance': 1, 'document_id': {0: 1}},
 'model': {'number of appearance': 21,
  'document_id': {0: 1,
   1: 1,
   6: 1,
   33: 1,
   35: 1,
   39: 1,
   41: 1,
   46: 1,
   48: 1,
   88: 1,
   97: 1,
   100: 1,
   101: 1,
   127: 1,
   136: 1,
   156: 1,
   157: 1,
   181: 1,
   185: 2,
   187: 1,
   203: 1}},
 'characteris': {'number of appearance': 3,
  'document_id': {0: 1, 27: 1, 111: 1}},
 'flower': {'number of appearance': 1, 'document_id': {0: 1}},
 'time': {'number of appearance': 5,
  'document_id': {0: 1, 26: 1, 105: 1, 188: 1, 203: 1}},
 'differ': {'number of appearance': 3, 'document_id': {0: 1, 198: 1, 216: 1}},
 'light': {'number of appearance': 3, 'document_id': {0: 1, 130: 2, 183: 1}},
 'qualiti': {'number of appearance': 1, 'document_id': {0: 1}},
 'co

In [ ]:
tokens = list(index.keys())
print(tokens)

['extend', 'plant', 'circadian', 'clock', 'model', 'characteris', 'flower', 'time', 'differ', 'light', 'qualiti', 'condit', 'challeng', 'prospect', 'climat', 'chang', 'impact', 'assess', 'mangrov', 'environ', 'mathemat', 'diabet', 'retinopathi', 'detect', 'use', 'transfer', 'reinforc', 'learn', 'effect', 'imag', 'preprocess', 'data', 'augment', 'techniqu', 'lamarckian', 'particl', 'swarm', 'optim', 'flexibl', 'ligand', 'dock', 'extract', 'evolutionari', 'backbon', 'scientif', 'domain', 'semant', 'main', 'path', 'network', 'approach', 'base', 'citat', 'context', 'analysi', 'gene', 'regulatori', 'infer', 'link', 'predict', 'graph', 'neural', 'impos', 'tidal', 'wave', 'finit', 'element', 'discontinu', 'galerkin', 'method', 'nonlinear', 'manifold', 'eeg', 'function', 'connect', 'applic', 'alzheim', 'diseas', 'born', 'woman', 'gender', 'robot', 'posenormnet', 'postur', 'normal', '3d', 'bodi', 'scan', 'arbitrari', 'unidirect', 'migrat', 'popul', 'alle', 'xdll', 'explain', 'deep', 'local', 'm

In [ ]:
# calculating the tf-idf score
def calculating_tfidf(index, data):

    N = len(data)

    for term, document_id in index.items():
        # th document frequncy (df) of the term
        df = document_id['number of appearance']
        # calculating the inverse document frequency (idf) of the term
        idf = math.log(N / df)
        # iterate over each document ID that the term appears in
        for docid, tf in document_id['document_id'].items():
            # calculate the term frequency-inverse document frequency (tf-idf) score of the term in the current document
            document_id['document_id'][docid] = tf * idf


    return index
indexer = (calculating_tfidf(index, data))
indexer

{'extend': {'number of appearance': 1, 'document_id': {0: 5.3981627015177525}},
 'plant': {'number of appearance': 2,
  'document_id': {0: 4.705015520957808, 33: 4.705015520957808}},
 'circadian': {'number of appearance': 1,
  'document_id': {0: 5.3981627015177525}},
 'clock': {'number of appearance': 1, 'document_id': {0: 5.3981627015177525}},
 'model': {'number of appearance': 21,
  'document_id': {0: 2.35364026379433,
   1: 2.35364026379433,
   6: 2.35364026379433,
   33: 2.35364026379433,
   35: 2.35364026379433,
   39: 2.35364026379433,
   41: 2.35364026379433,
   46: 2.35364026379433,
   48: 2.35364026379433,
   88: 2.35364026379433,
   97: 2.35364026379433,
   100: 2.35364026379433,
   101: 2.35364026379433,
   127: 2.35364026379433,
   136: 2.35364026379433,
   156: 2.35364026379433,
   157: 2.35364026379433,
   181: 2.35364026379433,
   185: 4.70728052758866,
   187: 2.35364026379433,
   203: 2.35364026379433}},
 'characteris': {'number of appearance': 3,
  'document_id': {0: 

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
# pre-processing the search word
def Cleaning_of_search_word(text):
  tokens = word_tokenize(text)
  token = [w.lower() for w in tokens if w.isalnum()]
  stopped_words = [w for w in token if w not in stopwords.words('english')]
  stemming = [stemmer.stem(w) for w in stopped_words]
  return stemming


In [ ]:
# creating search engine
import textwrap

def search(query, results_no=10, start=0):
    # preprocess the query to obtain tokens
    tokens = preprocess(query)
    results = {}
    # search for each token in the index
    for token in tokens:
        if token in index:
            # get the postings list for the token
            postings = index[token]['document_id']
            # update the score for each document in the results dictionary
            for docid, score in postings.items():
                if docid in results:
                    results[docid] += score
                else:
                    results[docid] = score

    # sort the results by score
    results = sorted(results.items(), key=lambda x: x[1], reverse=True)

    # display the results
    if len(results) == 0:
        print('publications not found.')
    else:
        print('publications results:')
        total_results = len(results)
        end = min(start + results_no, total_results)
        for i in range(start, end):
            docid, score = results[i]
            doc = data[docid]
            # print the title, score, link, date, abstract, and authors of the document
            print(f'{doc["title"]} ({score:.2f})')
            print(doc["link"])
            print(doc["date"])
            abstract = textwrap.fill(doc["abstract"], width=50)
            print(abstract)
            print(doc["authors"])
            print('\n'*2)

        # if there are more results, ask the user if they want to see them
        if end < total_results:
            next_input = input(f"Showing {start+1}-{end} of {total_results} results. Press 'n' to see more results.")
            if next_input.lower() == 'n':
                search(query, results_no=results_no, start=end)
query = input('insert search words: \n')
print("searching for publications")
search(query, results_no=10, start=0)

insert search words: 
cars and machine in computer technology
searching for publications
publications results:
Machine Learning for Computer Algebra (6.23)
https://pureportal.coventry.ac.uk/en/publications/machine-learning-for-computer-algebra
2022

[{'name': 'Barket, R.', 'href': 'https://pureportal.coventry.ac.uk/en/persons/rashid-barket'}, {'name': 'del Río, T.', 'href': 'https://pureportal.coventry.ac.uk/en/persons/tereso-del-r%C3%ADo-almajano'}, {'name': 'England, M.', 'href': 'https://pureportal.coventry.ac.uk/en/persons/matthew-england'}]



Stable likelihood computation for machine learning of linear differential operators with Gaussian processes (6.23)
https://pureportal.coventry.ac.uk/en/publications/stable-likelihood-computation-for-machine-learning-of-linear-diff
20 Apr 2022
In many applied sciences, the main aim is to learn
the parameters in the operational equations which
best fit the observed data. A framework for
solving such problems is to employ Gaussian
process (GP) 